In [1]:
import random

In [3]:
# Key Encoding:
# Total-AceAs11-Action
# (Total of Cards)-(0:No, 1:Yes)-(0:Stand, 1:Hit)

# Dict Value:
# [Counts, Probability]
class stateActionVal:
    def __init__(self):
        self.counts = {}
        for total in range(12, 21):
            key = str(total) + '-0-0'
            self.counts[key] = [0, 0]
            key = str(total) + '-0-1'
            self.counts[key] = [0, 0]
            key = str(total) + '-1-0'
            self.counts[key] = [0, 0]
            key = str(total) + '-1-1'
            self.counts[key] = [0, 0]

    def incCount(self, key):
        self.counts[key][0] += 1

    def updProb(self, key, outcome):
        self.counts[key][1] += outcome 

In [2]:
class player:
    def __init__(self):
        self.cards = []
        self.total = 0
        self.hasAce = False
    
    def addCard(self, card):
        self.cards.append(card)
        if(card.rank == "J" or card.rank == "Q" or card.rank == "K"):
            self.total += 10
        elif(card.rank == "A" and self.hasAce == False):
            self.total += 11
            self.hasAce = True
        else:
            if card.rank == 'A' and self.hasAce:
                self.total += 1
            else:
                self.total += int(card.rank)
    
    def hit(self, myDeck):
        self.addCard(myDeck.getRandomCard())
    
    
    def clear(self):
        self.cards = []
        self.total = 0
        self.hasAce = False

In [3]:
class card:
    def __init__(self, suit, rank):
        self.suit = suit # ♧, ♢, ♥, ♤
        self.rank = rank # 2, 3, 4, 5, 6, 7, 8, 9, 10, J, Q, K, A

In [4]:
class deck:
    def __init__(self):
        self.cards = []
        self.setDeck()
        
    def setDeck(self):
        suits = ["♧", "♢", "♥", "♤"]
        ranks = [2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K", "A"]

        for suit in suits:
            for rank in ranks:
                self.cards.append(card(suit, rank))  
    
    def printDeck(self):
        for card in self.cards:
            print(card.rank, card.suit)
    
    def shuffle(self):
        random.shuffle(self.cards)

    def getRandomCard(self):
        del self.cards[0]
        return self.cards[0]

In [20]:
class blackJackRound:
    def __init__(self, deck, player, dealer):
        self.player = player
        self.dealer = dealer
        self.deck = deck
        self.newRound()
        
    
    def newRound(self):
        self.player.clear()
        self.dealer.clear()
        self.deck.setDeck()
        self.deck.shuffle()
        self.hitPlayerBool = True

        self.player.hit(self.deck)
        self.player.hit(self.deck)
        self.dealer.hit(self.deck)
    
    def hitDealer(self):
        if(self.player.total <= 21): 
            while(self.dealer.total < 17):
                self.dealer.hit(self.deck)
    
    def hitPlayer(self):
        while(self.hitPlayerBool):
            if(self.player.total == 21):
                self.hitPlayerBool = False
            elif(self.player.total < 12):
                #Ai go hererererere
                self.player.hit(self.deck)
            else:
                self.hitPlayerBool = False
    
    def outcome(self):
        print("Player: ", self.player.total)
        print("Dealer: ", self.dealer.total)

        if(self.player.total <= 21 and (self.player.total > self.dealer.total or self.dealer.total > 21)):
            print("Player wins")
        elif(self.player.total == self.dealer.total):
            print("Draw")
        else:
            print("Dealer wins")
    
    def playRound(self):
        self.newRound()
        self.hitPlayer()
        self.hitDealer()
        self.outcome()

In [ ]:
class monteCarloControl:
    def __init__(self):
        self.tracker = stateActionVal()

    def exec(self, numLoop):
        for k in range((numLoop + 1)):
            continue

In [ ]:
class SARSA:
    def __init__(self):
        self.stateActionVal = stateActionVal()
    
    def training(self, numOfEps, player):
        learningFactor = 0.1
        discountFactor = 1

        for k in range(1, numOfEps):
            e = 1/k
            sa = epsilonGreedy([player.total, player.hasAce], e, self.stateActionVal)
            while player.total < 21:
                # Execute action an state s
                if sa[-1] == 1:
                    player.hit()
                else:
                    self.stateActionVal[sa][0] += 1
                    break
                
                # Get reward n form s`
                if player.total < 22:
                    n = 10 - (21 - player.total)
                else:
                    n = 21 - player.total

                # Get next action a`
                a = epsilonGreedy([player.total, player.hasAce], e, self.stateActionVal)

                # Get Q(s`, a`) from table
                saPrime = self.stateActionVal[a][1] # 0
                
                # update
                self.stateActionVal[sa][0] += 1
                self.stateActionVal[sa][1] = self.stateActionVal[sa][1] + learningFactor*(n + discountFactor*saPrime - self.stateActionVal[sa][1])

                #seting new sa
                sa = saPrime

In [ ]:
def epsilonGreedy(state, epsilonVal, stateAcVal): #state will be and array [total, hasAce]
        a1 = stateAcVal.counts[state[0]+"-"+state[1]+"-0"]
        a2 = stateAcVal.counts[state[0]+"-"+state[1]+"-1"]

        if a1[1] < a2[1]:
            worseOption = state[0]+"-"+state[1]+"-0"
            bestOption = state[0]+"-"+state[1]+"-1"
        else:
            worseOption = state[0]+"-"+state[1]+"-1"
            bestOption = state[0]+"-"+state[1]+"-0"

        if random.uniform(0, 1) < epsilonVal:
            return worseOption
        else:
            return bestOption

In [6]:
myDeck = deck()
myPlayer = player()
dealer = player()

BJ = blackJackRound(myDeck, myPlayer, dealer)

In [23]:
BJ.playRound()

Player:  21
Dealer:  10
Player wins
